In [81]:
#Importer vigtige pakker
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
import pandas_datareader.data as web
import datetime
import pydst
dst = pydst.Dst(lang='en')

In [82]:
# a. loading data
dst.get_variables('NABP10') # command to se 'id'
dst.get_variables('NABP10').iloc[0]['values'] # command to se 'values' in 'TRANSAKT'

# Vi henter kun nominelle priser, da "Aflønning af ansatte" ikke er angivet i faste priser 
gva10 = dst.get_data(table_id = 'NABP10', variables={'TRANSAKT':['B1GD','D1D'],'BRANCHE':['*'], 'PRISENHED':['V'], 'TID':['*']})
emp10 = dst.get_data(table_id = 'NABB10', variables={'SOCIO':['EMPH_DC','SALH_DC'],'BRANCHE':['*'], 'TID':['*']})

In [83]:
# b. define dictionaries to rename variables
columns_dict = {}
columns_dict['TRANSAKT'] = 'variable'
columns_dict['BRANCHE'] = 'industry'
columns_dict['PRISENHED'] = 'unit'
columns_dict['TID'] = 'year'
columns_dict['INDHOLD'] = 'value'
columns_dict['SOCIO'] = 'employment'

var_dict = {} # var is for variable
var_dict['B.1g Gross value added'] = 'GVA'
var_dict['D.1 Compensation of employees'] = 'ComEmp'
var_dict['Hours worked (1,000 hours)'] = 'Hours_TOT'
var_dict['Hours worked for employees (1,000 hours)'] = 'Hours_Emp'

unit_dict = {}
unit_dict['2010-prices, chained values'] = 'real'
unit_dict['Current prices'] = 'nominal'

# b. i
gva10.rename(columns=columns_dict,inplace=True)
emp10.rename(columns=columns_dict,inplace=True)

for key,value in var_dict.items():
    gva10.variable.replace(key,value, inplace=True)
    emp10.employment.replace(key,value, inplace=True)

for key,value in unit_dict.items():
    gva10.unit.replace(key,value, inplace=True)

# b. ii redefine value from 'str' to 'float'
emp10.value = emp10.value.astype('float')
gva10['value'] = gva10['value'].apply(pd.to_numeric, errors='coerce')

# skaber nye kolloner
emp10['Hours_TOT'] = emp10.value[emp10.employment == 'Hours_TOT']
emp10['Hours_Emp'] = emp10.value[emp10.employment == 'Hours_Emp']
gva10['GVA'] = gva10.value[gva10.variable == 'GVA']
gva10['ComEmp'] = gva10.value[gva10.variable == 'ComEmp']

gva10.tail()

,variable,industry,unit,year,value,GVA,ComEmp
1585,ComEmp,"R_S Arts, entertainment and other services",nominal,2014,38606,NaN,38606.0
1586,ComEmp,"R_S Arts, entertainment and other services",nominal,2015,38904,NaN,38904.0
1587,ComEmp,"R_S Arts, entertainment and other services",nominal,2016,40014,NaN,40014.0
1588,ComEmp,"R_S Arts, entertainment and other services",nominal,2017,41235,NaN,41235.0
1589,ComEmp,"R_S Arts, entertainment and other services",nominal,2018,42491,NaN,42491.0


In [84]:
# dropper 'employment'
emp10.drop(columns=['employment'], inplace = True)
emp10.drop(columns=['value'], inplace = True)
gva10.drop(columns=['variable'], inplace = True)
gva10.drop(columns=['value'], inplace = True)

In [94]:
 
#emp10 = emp10.dropna()
#gva10.head()
emp10.tail()
gva10.tail()

,industry,unit,year,GVA,ComEmp


In [86]:
# c. merged data
merged = pd.merge(gva10,emp10,how='left',on=['year','industry'])
merged

,industry,unit,year,GVA,ComEmp,Hours_TOT,Hours_Emp
0,Total,nominal,1966,73555.0,NaN,4604383.0,NaN
1,Total,nominal,1966,73555.0,NaN,NaN,3637974.0
2,Total,nominal,1967,80538.0,NaN,4537166.0,NaN
3,Total,nominal,1967,80538.0,NaN,NaN,3605118.0
4,Total,nominal,1968,88782.0,NaN,4467413.0,NaN
5,Total,nominal,1968,88782.0,NaN,NaN,3568827.0
6,Total,nominal,1969,101289.0,NaN,4453350.0,NaN
7,Total,nominal,1969,101289.0,NaN,NaN,3585142.0
8,Total,nominal,1970,112399.0,NaN,4449231.0,NaN
9,Total,nominal,1970,112399.0,NaN,NaN,3608565.0
